In [12]:
pip install pandas requests



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
print(df.columns)


Index(['Title', 'Field1'], dtype='object')


In [14]:
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import os
from urllib.parse import urlparse

def setup_session():
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

def download_pdf(link, folder_path, session):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
    }
    
    try:
        response = session.get(link, headers=headers, stream=True)
        # Check if the response is a PDF by content type header
        if 'application/pdf' in response.headers.get('Content-Type', '').lower():
            file_name = os.path.basename(urlparse(link).path)
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192): 
                    if chunk:  # filter out keep-alive new chunks
                        f.write(chunk)
            print(f"File downloaded successfully: {file_path}")
        else:
            print(f"Content type is not PDF: {response.headers.get('Content-Type', '')}, Link: {link}")

    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}, Link: {link}")

# Set your Excel file path and the folder path where PDFs should be saved
excel_path = '/Users/xiao1/Desktop/selected_pdfs_119.xlsx'  # Replace with the actual path to your Excel file
folder_path = '/Users/xiao1/Desktop/AICP modelling:Script/Modeltesting'  # Replace with your desired path

# Ensure the folder exists
os.makedirs(folder_path, exist_ok=True)

# Create a web session
session = setup_session()

# Read the Excel file
df = pd.read_excel(excel_path)

# Iterate over each link in the 'Field1' column and download it
for link in df['Field1'].dropna():  # This skips over NaN values
    download_pdf(link, folder_path, session)


File downloaded successfully: /Users/xiao1/Desktop/AICP modelling:Script/Modeltesting/npp%28en.pdf
Request error: HTTPSConnectionPool(host='sscp2pspc.ssc-spc.gc.ca', port=443): Max retries exceeded with url: /page.aspx/en/fil/download_public/D639616A-B5A9-417D-8E84-D6FE4D8D2C57 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)'))), Link: https://sscp2pspc.ssc-spc.gc.ca/page.aspx/en/fil/download_public/D639616A-B5A9-417D-8E84-D6FE4D8D2C57
File downloaded successfully: /Users/xiao1/Desktop/AICP modelling:Script/Modeltesting/r-154037-english-solicitation-amendment-1.pdf
Content type is not PDF: application/octet-stream, Link: https://canadabuys.canada.ca/documents/pub/att/2024/01/08/dffaafd874832eabc7967e50e9e2b639/ABES.PROD.PW_XYZ.B002.E29229.EBSU007.PDF
Content type is not PDF: application/octet-stream, Link: https://canadabuys.canada.ca/documents/pub/att/2024/01/08/d776456113f

KeyboardInterrupt: 